In [4]:
import os
import sys
import time
import math
import glob

import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl

import PIL

### data loading

In [7]:
data_dir = "../ranzcr-clip-catheter-line-classification/"

train_dir_path = os.path.join(data_dir, 'train')
test_dir_path = os.path.join(data_dir, 'test')
train_csv_path = os.path.join(data_dir, 'train.csv')
train_annotations_path = os.path.join(data_dir, 'train_annotations.csv')
sample_submission_file_path = os.path.join(data_dir, 'sample_submission.csv')

In [2]:
!ls ../ranzcr-clip-catheter-line-classification/ -lt

total 11688
drwxrwxr-x 2 welcome welcome    4096 Dec 26 23:05 train_tfrecords
drwxrwxr-x 2 welcome welcome 3375104 Dec 26 23:05 train
drwxrwxr-x 2 welcome welcome    4096 Dec 26 23:04 test_tfrecords
drwxrwxr-x 2 welcome welcome  393216 Dec 26 23:04 test
-rw-rw-r-- 1 welcome welcome 4950273 Dec 14 20:16 train_annotations.csv
-rw-rw-r-- 1 welcome welcome 2918266 Dec 14 20:02 train.csv
-rw-rw-r-- 1 welcome welcome  311839 Dec 14 19:59 sample_submission.csv


In [8]:
train = pd.read_csv(train_csv_path)


In [9]:
train.head()

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0,0,0,0,0,0,1,0,0,0,0,ec89415d1
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0,0,1,0,0,1,0,0,0,1,0,bf4c6da3c
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0,0,0,0,0,0,0,0,1,0,0,3fc1c97e5
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0,0,0,0,0,0,0,1,0,0,0,c31019814
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0,0,0,0,0,0,0,0,0,1,0,207685cd1


In [11]:
ls -lt ../ranzcr-clip-catheter-line-classification/train | head -n 10

total 6810572
-rw-rw-r-- 1 welcome welcome 312114 Dec 14 20:16 1.2.826.0.1.3680043.8.498.99971231542589897710744415581839623025.jpg
-rw-rw-r-- 1 welcome welcome 159343 Dec 14 20:16 1.2.826.0.1.3680043.8.498.99971919985113251695401367974124184826.jpg
-rw-rw-r-- 1 welcome welcome 163899 Dec 14 20:16 1.2.826.0.1.3680043.8.498.99974757496176821720048757591692208827.jpg
-rw-rw-r-- 1 welcome welcome 325595 Dec 14 20:16 1.2.826.0.1.3680043.8.498.99979645057411935093674597476197972475.jpg
-rw-rw-r-- 1 welcome welcome 169122 Dec 14 20:16 1.2.826.0.1.3680043.8.498.99989600335044826115371936809352740080.jpg
-rw-rw-r-- 1 welcome welcome 308266 Dec 14 20:16 1.2.826.0.1.3680043.8.498.99991581639229040366812735770671130168.jpg
-rw-rw-r-- 1 welcome welcome 361610 Dec 14 20:16 1.2.826.0.1.3680043.8.498.99704910154542866752812317447226576492.jpg
-rw-rw-r-- 1 welcome welcome 154911 Dec 14 20:16 1.2.826.0.1.3680043.8.498.99710332543782550669055201234440116474.jpg
-rw-rw-r-- 1 welcome welcome 153123 Dec 14

In [1]:
# import tensorflow as tf
#     
# raw_dataset = tf.data.TFRecordDataset("../ranzcr-clip-catheter-line-classification/train_tfrecords/00-1881.tfrec")
# 
# for raw_record in raw_dataset.take(1):
#     example = tf.train.Example()
#     example.ParseFromString(raw_record.numpy())
#     print(example)
#     break